## loading libraries and dataset...

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix
import shap
import matplotlib.pyplot as plt
import random, os
from sklearn.linear_model import LogisticRegression

# Loading and re-sorting for temporal order...
df = pd.read_csv("C:/Users/LATITUDE E7470/Desktop/ajcodes/research/fatigueanalysis1/eye_features.csv")
df = df.sort_values(by='image_name').reset_index(drop=True)

# Feature engineering
df['ear_rolling_mean'] = df['ear'].rolling(window=5, min_periods=1).mean()
df['ear_delta'] = df['ear'].diff().fillna(0)
df['eye_closed'] = df['ear'].apply(lambda x: 1 if x < 0.21 else 0)

# Choosing features for LSTM input
features = df[['ear', 'ear_rolling_mean', 'ear_delta', 'eye_closed']].values
labels = df['label'].values

# Generating sequences...
SEQ_LEN = 10
X_seq, y_seq = [], []

for i in range(len(features) - SEQ_LEN):
    X_seq.append(features[i:i+SEQ_LEN])
    y_seq.append(labels[i+SEQ_LEN - 1])  # use label of last frame

X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

print("Shape of sequence data:", X_seq.shape)  # (samples, time_steps, features)


## splitting into train/test set...

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_seq, y_seq, test_size=0.3, random_state=42, stratify=y_seq
)

## coding for LSTM Model...

In [ ]:
model = Sequential([
    Input(shape=(SEQ_LEN, X_seq.shape[2])),  # Defining input shape...
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

## evaluating the LSTM Model....

In [ ]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype("int32")

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
# Save the model
model.save("ear_lstm_model.h5")

## Reloading it cleanly outside training context...

In [ ]:
from keras.models import load_model
model = load_model("ear_lstm_model.h5")

## implementing SHAP for LSTM...

In [ ]:
# Use SHAP safely with permutation explainer
explainer = shap.Explainer(model.predict, X_train[:50], algorithm="permutation")
shap_values = explainer(X_test[:10])  # keep it small

shap.plots.beeswarm(shap_values, feature_names=['EAR', 'Rolling Mean', 'Delta EAR', 'Eye Closed'])